# RGB and CIE
In this project we'll regularly need to calculate distances between pixels in colour space as a proxy for the visual difference between the colours. The simplest way of doing this is to calculate the Euclidean distance between them (also known as cosine distance or $\ell2$ norm).
If we have two colours $C_1 = (R_1, G_1, B_1)$ and $C_2 = (R_2, G_2, B_2)$, the Euclidean distance $\Delta C$ is defined as:

${\displaystyle \Delta C ={\sqrt {(R_{2}-R_{1})^{2}+(G_{2}-G_{1})^{2}+(B_{2}-B_{1})^{2}}}}$

We can implement the function in python as follows

In [ ]:
def colour_distance_1(colour_1, colour_2):
    return sum([(channel_2 - channel_1) ** 2 
                for channel_1, channel_2 in 
                zip(colour_1, colour_2)]) ** 0.5

The red, green, and blue channels available to us in RGB space are ideally suited for representing colour on pixelated screens. However, our goal is to represent the _ perceptual differences_ between colours, and RGB isn't ideal for this. It's now [pretty well established](https://en.wikipedia.org/wiki/Color_difference) that euclidean distances in RGB space are a bad representation of the distances that our eyes see.
By stretching the RGB dimensions by different amounts, we can better approximate that difference:

$\displaystyle \Delta C ={ {\sqrt {2\times \Delta R^{2}+4\times \Delta G^{2}+3\times \Delta B^{2}}}}$

Again, here's the python

In [ ]:
def colour_distance_2(colour_1, colour_2):
    r_1, g_1, b_1 = colour_1
    r_2, g_2, b_2 = colour_2

    return (2 * (r_1 - r_2) ** 2 +
            4 * (g_1 - g_2) ** 2 +
            3 * (b_1 - b_2) ** 2) ** 0.5

We can improve further by adding some extra weirdness to the red and blue channels

${\displaystyle \Delta C={\sqrt {2\times \Delta R^{2}+4\times \Delta G^{2}+3\times \Delta B^{2}+{{{\bar {r}}\times (\Delta R^{2}-\Delta B^{2})} \over {256}}}}}$

Here it is in python

In [ ]:
def colour_distance_3(colour_1, colour_2):
    r_1, g_1, b_1 = colour_1
    r_2, g_2, b_2 = colour_2
    
    d_r_sq = (r_1 - r_2) ** 2
    d_g_sq = (g_1 - g_2) ** 2
    d_b_sq = (b_1 - b_2) ** 2
    mean_r = (r_1 + r_2) / 2
    
    d_c_sq = (2 * d_r_sq + 4 * d_g_sq + 3 * d_b_sq +
              (mean_r * (d_r_sq - d_b_sq) / 256))

    return d_c_sq ** 0.5

The most general and efficient approach (as far as I know) is to transform the image's RGB coordinates into an entirely new space. The _International Commission on Illumination_ (CIE) produced [CIELAB](https://en.wikipedia.org/wiki/CIELAB_color_space#CIELAB) to better approximate human perception of colour distances. The three coordinates of CIELAB represent:

- The lightness of the color. `L` = 0 yields black and `L` = 100 indicates diffuse white.
- its position between red/magenta and green (`a`, negative values indicate green while positive values indicate magenta)
- its position between yellow and blue (`b`, negative values indicate blue and positive values indicate yellow).

[CIE76](https://en.wikipedia.org/wiki/Color_difference#CIE76) (ie euclidean distance in LAB space) was the original distance proposed with the space. It's been improved upon since, but the differences are minor and as far as I've seen, are unnecessary complications for such minimal gain.

We can map from RGB to LAB and back again by importing the relevant function from `skimage`

In [ ]:
import numpy as np
from skimage.color import rgb2lab, lab2rgb

In this new space we can use our first, super-simple colour distance function to measure the perceptual difference between colours. Below we're randomly generating two colours, converting them to LAB space and calculating the distance. This distance can be seen as a kind of inverse similarity score (colour pairs with lower distance values are more perceptually similar).

In [ ]:
rgb_colour_1 = np.random.randint(0, 255, (1, 1, 3)).astype(np.float64)
rgb_colour_2 = np.random.randint(0, 255, (1, 1, 3)).astype(np.float64)

lab_colour_1 = rgb2lab(rgb_colour_1).squeeze()
lab_colour_2 = rgb2lab(rgb_colour_2).squeeze()

colour_distance_1(lab_colour_1, lab_colour_2)